In [ ]:
%matplotlib inline
import cv2
import dlib
import numpy as np
import matplotlib.pyplot as plt
import os
from subprocess import call


def get_landmarks(img):
    faces = detector(img, 1)

    if len(faces) < 1:
        return 'Error'

    bb = max(faces, key=lambda rect: rect.width() * rect.height())
    points = predictor(img, bb)
    landmarks = map(lambda p: (p.x, p.y), points.parts())

    return landmarks


def get_mutated_image(img, part, color):
    img = img.copy()
    landmarks = get_landmarks(img)
    if landmarks == 'Error':
        return 'Error'

    pts = []

    ### left eye ###
    if part == 'leye':
        left_top     = (landmarks[36][0], min(landmarks[37][1], landmarks[38][1]))
        right_bottom = (landmarks[39][0], max(landmarks[40][1], landmarks[41][1]))
        cv2.rectangle(img, left_top, right_bottom, color, -1)
        cv2.rectangle(img, left_top, right_bottom, color, 12)

    ### right eye ###
    if part == 'reye':
        left_top     = (landmarks[42][0], min(landmarks[43][1], landmarks[44][1]))
        right_bottom = (landmarks[45][0], max(landmarks[46][1], landmarks[47][1]))
        cv2.rectangle(img, left_top, right_bottom, color, -1)
        cv2.rectangle(img, left_top, right_bottom, color, 12)

    ### nose ###
    if part == 'nose':
        pts.append([landmarks[32][0], landmarks[27][1]])
        pts.append([landmarks[34][0], landmarks[27][1]])
        pts.append([landmarks[31][0], landmarks[33][1]])
        pts.append([landmarks[35][0], landmarks[33][1]])
        hull = cv2.convexHull(np.array(pts))
        cv2.drawContours(img, [hull], -1, color, -1)
        cv2.drawContours(img, [hull], -1, color, 12)

    ### mouth ###
    if part == 'mouth':
        mouth_idx = [48, 50, 52, 54, 55, 56, 57, 58, 59]

        for idx in mouth_idx:
            pts.append([landmarks[idx][0], landmarks[idx][1]])
        hull = cv2.convexHull(np.array(pts))

        cv2.drawContours(img, [hull], -1, color, -1)
        cv2.drawContours(img, [hull], -1, color, 12)

    return img


def get_attri_pts(lms, attri):
    pts = []
    if attri == 'leye':
        pts.append([lms[36][0], min(lms[37][1], lms[38][1])])
        pts.append([lms[39][0], min(lms[37][1], lms[38][1])])
        pts.append([lms[36][0], max(lms[40][1], lms[41][1])])
        pts.append([lms[39][0], max(lms[40][1], lms[41][1])])
    elif attri == 'reye':
        pts.append([lms[42][0], min(lms[43][1], lms[44][1])])
        pts.append([lms[45][0], min(lms[43][1], lms[44][1])])
        pts.append([lms[42][0], max(lms[46][1], lms[47][1])])
        pts.append([lms[45][0], max(lms[46][1], lms[47][1])])
    elif attri == 'nose':
        pts.append([lms[32][0], lms[27][1]])
        pts.append([lms[34][0], lms[27][1]])
        pts.append([lms[31][0], lms[33][1]])
        pts.append([lms[35][0], lms[33][1]])
    elif attri == 'mouth':
        pts.append([lms[48][0], min(lms[50][1], lms[52][1])])
        pts.append([lms[64][0], min(lms[50][1], lms[52][1])])
        pts.append([lms[48][0], lms[57][1]])
        pts.append([lms[64][0], lms[57][1]])
    return pts


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('../data/shape_predictor_68_face_landmarks.dat')

In [ ]:
### show landmarks ###
base_img = cv2.imread('../data/base_img.jpg')
base_lms = get_landmarks(base_img)

img = cv2.cvtColor(base_img, cv2.COLOR_BGR2RGB)
for (x, y) in base_lms:
    cv2.circle(img, (x, y), 2, (0, 150, 255), -1)

plt.imshow(img)
plt.show()

In [ ]:
### attribute mutation ###
attributes = ['leye', 'reye', 'nose', 'mouth']
actions    = ['substitution', 'preservation']

attri = attributes[0]
act   = actions[0]

base_img      = cv2.imread('../data/base_img.jpg')
base_lms      = get_landmarks(base_img)
base_no_attri = get_mutated_image(base_img, attri, (0,0,0))
base_attri    = cv2.subtract(base_img, base_no_attri)

count = 0
for img_name in os.listdir('../data/images'):
    img_name = img_name.strip()
    print count, img_name
    count += 1

    other_img      = cv2.imread('../data/images/' + img_name)
    other_lms      = get_landmarks(other_img)
    other_no_attri = get_mutated_image(other_img, attri, (0,0,0))
    other_attri    = cv2.subtract(other_img, other_no_attri)

    base_pts  = get_attri_pts(base_lms, attri)
    other_pts = get_attri_pts(other_lms, attri)

    if act == 'substitution':
        trans_src = other_pts
        trans_dst = base_pts
        attri_src = other_attri
        bg_src    = base_img
    else:
        trans_src = base_pts
        trans_dst = other_pts
        attri_src = base_attri
        bg_src    = other_img

    trans      = cv2.getPerspectiveTransform(np.float32(trans_src), np.float32(trans_dst))
    new_attri  = cv2.warpPerspective(attri_src, trans, (224,224))
    gray_attri = cv2.cvtColor(new_attri, cv2.COLOR_BGR2GRAY)
    ret, mask  = cv2.threshold(gray_attri, 10, 255, cv2.THRESH_BINARY)
    mask_inv   = cv2.bitwise_not(mask)
    background = cv2.bitwise_and(bg_src, bg_src, mask=mask_inv)
    foreground = cv2.bitwise_and(new_attri, new_attri, mask=mask)

    replaced_img = cv2.add(background, foreground)

    ### show attribute-replaced image ###
    output = cv2.cvtColor(replaced_img, cv2.COLOR_BGR2RGB)
    plt.imshow(output)
    plt.show()

    ### save attribute-replaced image ###
    out_path = '../data/attribute_mutated/'+ attri + '_' + act
    call(['mkdir', out_path])
    cv2.imwrite(out_path + '/' + img_name, replaced_img)